# 랭체인을 이용한 RAG 챗봇

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
# from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
# from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
import urllib.request
import gradio as gr

/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")

('2020_경제금융용어 700선_게시.pdf', <http.client.HTTPMessage at 0x7f580f8f0220>)

In [4]:
loader = PyPDFLoader("2020_경제금융용어 700선_게시.pdf")
texts = loader.load_and_split()

In [5]:
print('문서의 수 :', len(texts))

문서의 수 : 366


In [6]:
# 0번 문서는 머리말
print(texts[0].page_content)

iii
찾아보기
한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인 
의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠 
개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 . 
이의 일환으로 2018년 
 경제금융용어 700선
 을 발간하였는데 그간 동 책자에 대한 
수요가 꾸준히 늘어남에 따라 이번에 추가로 발간하게 되었습니다 .
지난번 내용과 같이 통화정책, 실물경제, 금융안정, 지급결제 등 한국은행 주요 
업무를 이해하는데 필요한 전문 용어와 경제·금융 흐름을 이해하는데 도움이 되는 
시사 경제금융 용어들을 수록하였습니다. 용어해설은 개념과 도입 배경, 의미, 적용 
사례 등을 담아 쉽게 이해할 수 있도록 하였습니다. 또한 e-book으로도 제작하여 
독자들의 편의성과 가독성을 높였습니다 . 
코로나 19로 이전에 경험하지 못한 경제위기를 겪으면서 경제·금융 현상에 대한 
일반인들의 관심도가 이전에 비해 높아졌습니다. 아무쪼록 이 책자가 한국은행의 주요 
정책과 국내외 경제를 이해하는 데 조금이나마 도움이 되기를 바라겠습니다. 끝으로 
경제금융용어 700선
  집필에 참여해주신 경제교육실 교수님들과 용어 집필에 좋은 
의견을 주신 관련부서 여러분께 감사드립니다 .
2020년 8월
한국은행 경제교육실장 박 철 원
머 리 말


In [7]:
# 12번 문서까지는 목차
print(texts[12].page_content)

xiii
찾아보기❙
환매조건부매매/RP/Repo ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・331
환 어 음 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・332
환율조작국 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・332
환전영업자(환전상) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・333
환차손/환차익 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・333
황금낙하산 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・334
회 사 채 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・334
회수의문 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후방연쇄효과 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후순위금융채 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후행종합지수 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・336
ABC
B e y o n d  G D P ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・337
BIS 자기자본비율 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・338
CAMEL-R시스템/ROCA시스템/
C A C R E L 시 스 템 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・338
C L S 은 행 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・339
C M O ・・・・・・・・・・・・・・・・

In [8]:
# 13번 문서부터는 금융 용어 설명
print(texts[13].page_content)

1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


### 0번 청크부터 12번 청크까지는 필요하지 않으므로 제거하자

In [9]:
texts = texts[13:]
print('줄어든 texts의 길이 :', len(texts))

줄어든 texts의 길이 : 353


In [10]:
print(texts[-1])

page_content='경제금융용어 700선
발행인 이주열
편집인 박철원
발행처 한국은행(www.bok.or.kr)
서울특별시 중구 세종대로 67(태평로 2가)
발행일 2020년 8월 3일
인  쇄 ㈜제일프린테크 
 본 자료는 한국은행 홈페이지(http://www.bok.or.kr>경제교육>온라인학습>일반인) 에서 
다운로드 받으실 수 있습니다.
 이 책자에 대한 문의는 한국은행 경제교육실 경제교육기획팀(02-759-5618)으로 연락 
하여 주시기 바랍니다.
 
 경제금융용어 700선
 은 
① 정부간행물판매센터(☎ 02-734-6818, http://www.gpcbooks.co.kr) 
② 한국경제서적(☎ 02-737-7498)
③ 경제서적(☎ 02-736-0640, http://kj-book.co.kr) 또는 주요 서점 등에서 매권당 
8,000원에 판매하고 있습니다.
ISBN 979-11-5538-393-3 03320' metadata={'source': '2020_경제금융용어 700선_게시.pdf', 'page': 369}


In [11]:
print(texts[-2])

page_content='352
경제금융용어 700선
와 관련된 다양한 부수업무를 하는 회사를 말한다. 가령 신용카드 거래가 발생하면 
VAN사업자는 거래 승인을 위해 신용카드사에 거래내역을 전송하고 승인 및 조회결과를 
수신한 후 이를 가맹점에 전송한다. 또한 VAN사업자는 가맹점이 신용카드사에 전표를 
제출하여 대금을 청구하는 업무를 대신하기 위하여 가맹점을 방문하여 전표실물을 수거
하거나 전표 데이터를 수집한다 . 
 연관검색어 : 지급결제시스템, 전자금융
VaR(Value at Risk) 
주어진 신뢰수준 하에서 일정 기간 동안 발생할 수 있는 ‘최대 손실금액’으로 금융기관
의 잠재적인 손실을 측정하는 지표이다. 예를 들어 목표기간 1년, 신뢰수준 95% 에서 
산출된 VaR가 10억 원이라면 이는 1년 동안 발생할 수 있는 손실금액이 10억 원보다 
작을 확률이 95%라는 것을 의미한다 . 
 연관검색어 : 예상손실
VIX
미국 주식시장의 단기 변동성에 대한 시장의 기대치를 나타내는 지수로 시카고옵션거
래소(CBOE)에서 제공되며, 정식명칭은 CBOE Volatility Index이다. VIX는 향후 30 일 
동안의 S&P 500 지수의 변동성에 대한 시장의 기대치로서, 지수의 변동성이 클 것으로 
예상될 경우 옵션가격이 높아지는 점에 착안하여 CBOE에 상장된 다양한 행사가격의 
S&P 500 지수 옵션들의 가격을 활용하여 산출된다. VIX는 일반적으로 기초자산 가격과 
음(-)의 상관관계가 있다. 예를 들어 주가지수가 상승할 때 하락하고 주가지수가 하락할 
때는 상승한다. 이에 따라 VIX의 상승은 투자자들의 불안심리가 증대하는 것을 의미하므
로 공포지수라고도 부른다. 1993년 Robert E. Whaley 교수의 논문에서 처음 소개된 
이후 시장의 변동성과 투자자들의 심리를 나타내주는 주요한 지표로 널리 활용되고 
있다. 이 외의 대표적인 주식시장 변동성 지수로는 유럽의 VSTOXX 등이 있다. VSTOXX
는 유럽의 대표 주가지수인 Euro

### 마지막 청크는 이 문서의 맺음말에 해당되므로 해당 데이터도 불필요하므로 제거

In [12]:
# 마지막 데이터를 제거
texts = texts[:-1]
print('마지막 데이터 제거 후 texts의 길이 :', len(texts))

마지막 데이터 제거 후 texts의 길이 : 352


### 이제 임베딩

In [13]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory = './chroma_db')

In [14]:
# 벡터DB의 개수 확인
vectordb._collection.count()

352

In [15]:
for key in vectordb._collection.get():
  print(key)

ids
embeddings
metadatas
documents


In [16]:
# embedding 호출 시도
result = vectordb._collection.get()['embeddings']
print(result)

None


In [17]:
# embedding vetor만 조회하기
embeddings = vectordb._collection.get(include=['embeddings'])['embeddings']
print('임베딩 벡터의 개수 :', len(embeddings))

임베딩 벡터의 개수 : 352


In [18]:
print('첫번째 문서의 임베딩 값 출력 :', embeddings[0])
print('첫번째 문서의 임베딩 값의 길이 :', len(embeddings[0]))

첫번째 문서의 임베딩 값 출력 : [-0.0076664709486067295, -0.007309571374207735, 0.014495600946247578, -0.045353662222623825, -0.0076664709486067295, 0.014440692961215973, -0.0037577375769615173, 0.005473599303513765, -0.00681197177618742, -0.027893049642443657, 0.004869616124778986, 0.033383809030056, -0.020096173509955406, 0.01218948233872652, 0.0012174040311947465, 0.02757733128964901, 0.012669923715293407, -0.01824304275214672, 0.020096173509955406, -0.03541538864374161, -0.015099584124982357, -0.005223083309829235, -0.03316417708992958, 0.0012971916003152728, -0.0017947915475815535, 0.00404943386092782, 0.027343975380063057, -0.01171590480953455, -0.009643143974244595, 0.011475684121251106, 0.007268390618264675, -0.02023344300687313, -0.012065940536558628, -0.017309613525867462, -0.0021396796219050884, 0.014632869511842728, 0.005020611919462681, 0.02145513705909252, 0.02288273349404335, -0.011159965768456459, 0.0033407830633223057, 0.007055623922497034, -0.004756369162350893, 0.0203844383358955

In [19]:
metadatas = vectordb._collection.get()['metadatas']
print('metadatas의 개수 :', len(metadatas))
print('첫번째 문서의 출처 :', metadatas[0])

metadatas의 개수 : 352
첫번째 문서의 출처 : {'page': 17, 'source': '2020_경제금융용어 700선_게시.pdf'}


In [20]:
# 유사도가 높은 문서 2개만 추출. k = 2
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [21]:
docs = retriever.get_relevant_documents("비트코인이 궁금해")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('두번째 유사 문서 :', docs[1])

/tmp/ipykernel_1631652/1534360734.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("비트코인이 궁금해")


유사 문서 개수 : 2
----------------------------------------
첫번째 유사 문서 : page_content='139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다. 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수량을 한정시
켜 놓았기 때문에 원칙적으로 인플레이션에 의해 가치가 떨어질 수 있는 화폐와 다

In [22]:
# Create Prompt
template = """당신은 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이입니다.
안상준 개발자가 만들었습니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)

In [23]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=retriever,
    return_source_documents=True)

In [25]:
input_text = "디커플링이란 무엇인가?"
chatbot_response = qa_chain.invoke(input_text)
print(chatbot_response)

{'query': '디커플링이란 무엇인가?', 'result': '디커플링은 탈동조화라고도 하는데, 어떤 나라나 지역의 경제가 인접한 다른 국가나 전반적인 세계 경제의 흐름과는 다른 모습을 보이는 현상을 말해. 예를 들어, 금융위기 이후 신흥국가나 유로지역 국가들이 미국 경제와 다른 모습을 보이는 경우가 있어. 주가나 금리, 환율 등 일부 경제 변수의 흐름이 국가 간 또는 특정 국가 내에서 서로 다른 흐름을 보이는 현상도 디커플링이라고 할 수 있어. 반대로, 경제가 다른 국가나 세계 경제 흐름과 유사한 흐름을 보이는 것을 커플링이라고 해.', 'source_documents': [Document(metadata={'page': 114, 'source': '2020_경제금융용어 700선_게시.pdf'}, page_content='98\n경제금융용어 700선\n가계와 기업이 소비와 투자를 미룸으로써 수요의 위축을 초래하여 디플레이션 압력을 \n보다 크게 할 수도 있다. 이 같은 상황에서는 경제주체들의 물가상승률 예상치가 더 \n낮아지지 않도록 사전에 통화 ･재정정책 면에서 적극적으로 대응하는 것이 중요하다 .\n 연관검색어 : 디플레이션,  통화정책\n디커플링/커플링\n디커플링(decoupling)이란 탈동조화라고 번역할 수 있는데, 어떤 나라나 지역의 경제\n가 인접한 다른 국가나 전반적인 세계 경제의 흐름과는 다른 모습을 보이는 현상을 \n말한다. 최근의 디커플링의 예로는 금융위기 이후 신흥국가나 유로지역 국가 등이 특히 \n미국 경제와 다른 모습을 보이는 것을 들 수 있다. 이외에도 주가나 금리, 환율 등 \n일부 경제 변수의 흐름이 국가 간 또는 특정국가내에서 서로 다른 흐름을 보이는 현상도 \n디커플링이라고 할 수 있다. 국가 간의 경우 미국의 주가와 한국의 주가가 다른 방향으로 \n움직이거나, 한 국가 내에서의 경우 주가가 하락함에도 해당국 통화가 강세 현상을 \n보이는 경우 등을 말한다. 반대로 한 나라 또는 지역의 경제가 인접한 다른 국가나 \n세계

In [26]:
input_text = "너는 누구야?"
chatbot_response = qa_chain.invoke(input_text)
print(chatbot_response)

{'query': '너는 누구야?', 'result': '나는 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이야. 안상준 개발자가 만들었어. 궁금한 금융 용어가 있으면 물어봐! 검색 결과에 있는 내용만 설명해줄 수 있어.', 'source_documents': [Document(metadata={'page': 253, 'source': '2020_경제금융용어 700선_게시.pdf'}, page_content='237\nㅈ \n자유재\n희소성이 높지 않고 아무런 대가를 치르지 않아도 무상으로 얻을 수 있는 재화를 \n말한다. 이는 희소성을 가지며 경제적 거래를 통해 얻을 수 있는 경제재 (economic \ngoods)의 반대 개념이다. 자유재(free goods)는 대가를 치르지 않고서도 얼마든지 원하\n는 만큼 얻을 수 있기 때문에 경제문제를 일으키지 않으며 희소성의 원칙도 적용되지 \n않는다. 현실세계에서 자유재는 매우 드물다. 공기, 물, 햇빛 등이 전통적으로 자유재로 \n고려되었으나 양질의 생수를 높은 비용으로 구매하고, 맑은 공기를 마시기 위하여 휴양림\n을 찾아가는 등 기존에 자유재로 고려되던 재화들이 상황 변화에 따라 경제재로 변하기도 \n한다. 일반적으로 재화라고 하면 경제재를 의미하며 통상 경제학은 경제재를 분석대상으\n로 한다 . \n 연관검색어 : 기펜재\n작업증명\n작업증명(proof-of-work)은 제3자로부터의 서비스 요청(request)이 악의적인 공격\n(spam 또는 DoS, Denial of Service)이 아님을 증명하기 위해서, 자원이 소모되는 작업을 \n추가로 수행할 것을 요구하는 것을 의미한다. 대표적인 작업증명으로는 웹사이트 로그인\n시 특정한 문자 및 그림을 읽고 입력할 것을 요구하는 캡차를 들 수 있다. 작업증명의 \n특징은 비대칭성인데, 서비스 요청자가 수행하는 데는 어느 정도 자원이 소모되지만 \n이를 확인하기는 매우 쉬워야 한다. 작업증명은 가상통화인 비트코인 ･이더리움 등을 \n채굴하는 합의 과정에 활용

In [27]:
input_text = "23년 9월달의 서울 집값 추이를 알려주세요"
chatbot_response = qa_chain.invoke(input_text)
print(chatbot_response)

{'query': '23년 9월달의 서울 집값 추이를 알려주세요', 'result': '미안한데, 23년 9월달의 서울 집값 추이에 대한 정보는 제공할 수 없어. 다른 금융 용어에 대해 궁금한 게 있으면 물어봐!', 'source_documents': [Document(metadata={'page': 65, 'source': '2020_경제금융용어 700선_게시.pdf'}, page_content='49\nㄱ \n금의 일시 보전을 위해 ｢국고금관리법｣에 의해 발행되는데 만기는 1년 이내지만 실제로\n는 3개월 이내이며 경쟁입찰방식으로 발행된다. 국민주택채권은 국민주택건설 재원조달\n을 위해 ｢주택도시기금법｣에 의해 부동산 등기 및 인허가와 관련하여 의무적으로 매입하\n도록 하는 첨가소화방식으로 발행되며 만기는 5년이다. 보상채권은 공공용지 보상비 \n마련을 위하여 ｢공익사업을 위한 토지 등의 취득 및 보상에 관한 법률｣에 의해 당사자앞 \n교부방식으로 발행되며 만기는 5년 이내지만 실제로는 3년 만기로 발행한다 . \n규모의 경제\n일반적인 경우 기업이 재화 및 서비스 생산량을 늘려감에 따라 추가적으로 소요되는 \n평균 생산비용도 점차 늘어난다. 그런데 일부 재화 및 서비스 생산의 경우에는 생산량이 \n늘어날수록 평균비용이 감소하는 현상이 나타나는데 이를 규모의 경제(economies of \nscale)라고 한다. 이런 현상은 초기 생산단계에서 막대한 투자비용이 필요하지만 생산에\n는 큰 비용이 들지 않는 철도･통신･전력산업에서 나타나는데, 이들 산업은 생산이 시작\n된 이후 수요가 계속 늘게 되면서 평균 생산비용도 감소하는 특징을 갖고 있다. 또한 \n분업에 따른 전문화 이익이 존재하는 경우에도 규모의 경제가 나타난다. 예를 들어 \n각자 케이크를 만드는 3명의 제빵사를 생각해보자. 이들이 따로 케이크를 만들 때 보다 \n한 명은 반죽만, 한 명은 굽기만, 한 명은 장식만 하는 분업을 할 경우 생산량이 늘어나면\n서 평균 비용도 줄게 된다 .\n 연관검색어

In [28]:
def get_chatbot_response(input_text):
    chatbot_response = qa_chain.invoke(input_text)
    return chatbot_response['result'].strip()

In [29]:
input_text = "너는 뭘하는 챗봇이니?"
result = get_chatbot_response(input_text)
print(result)

나는 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이야. 안상준 개발자가 만들었고, 주어진 검색 결과를 바탕으로 금융 용어에 대해 설명해줄 수 있어. 검색 결과에 없는 내용이라면 답변할 수 없어. 궁금한 게 있으면 물어봐!


In [30]:
input_text = "나는 보수당을 지지해야 한다고 생각해"
result = get_chatbot_response(input_text)
print(result)

미안한데, 보수당을 지지해야 한다는 생각에 대한 정보는 검색 결과에 없어서 답변할 수 없어. 다른 금융 관련 질문이 있으면 언제든지 물어봐!
